In [2]:
# 寫進文件當目錄

import requests
import yaml
import json
import pandas as pd


app_id = 'catalinakuowork-ec9c8282-4add-40b0'
app_key = '7d75e95f-082b-4076-ae4d-8f0ec033b56f'
auth_url = "https://tdx.transportdata.tw/auth/realms/TDXConnect/protocol/openid-connect/token"


def fetch_swagger_doc(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return yaml.safe_load(response.text)
    except requests.exceptions.RequestException as e:
        print(f'Error retrieving Swagger doc: {str(e)}')
        return None


def save_as_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)


def save_as_excel(data, filename):
    df = pd.DataFrame(data)
    df.to_excel(filename, index=False)


def main():
    # Fetch the Swagger doc
    # 获取pK_ServiceDetails
    url = "https://tdx.transportdata.tw/webapi/service?category=basic"
    response = requests.get(url)
    data = response.json()

    pK_ServiceDetails = [item['pK_ServiceDetail'] for item in data['list']]
    
    data_json = []
    data_excel = []

    for pK in pK_ServiceDetails:
        url = f"https://tdx.transportdata.tw/webapi/service/GetServiceDetail/{pK}"
        response = requests.get(url)
        data = response.json()

        data_url = data['dataUrl']
        if '{' in data_url:
            continue
        data_name = data['name']
        data_num = data['pK_ServiceDetail']
        data_desc = data["domainList"]
        data_freq = data["serviceUpdateFrequency"]

        # Save as JSON
        data_json.append({
            '資料集代碼': data_num,
            '資料集名稱': data_name,
            '資料集api': data_url,
            '資料集描述': data_desc,
            '資料集更新頻率': data_freq
        })

        # Save as Excel
        data_excel.append({
            '資料集代碼': data_num,
            '資料集名稱': data_name,
            '資料集描述': data_desc,
            '資料集api': data_url,
            '資料集更新頻率': data_freq
        })

        save_as_json(data_json, 'TDX運輸資料清單_3.json')
        save_as_excel(data_excel, 'TDX運輸資料清單_3.xlsx')
        
if __name__ == '__main__':
    main()


In [9]:
# to csv in folder
import requests
import os
import json
import csv

app_id = 'catalinakuowork-ec9c8282-4add-40b0'
app_key = '7d75e95f-082b-4076-ae4d-8f0ec033b56f'

auth_url = "https://tdx.transportdata.tw/auth/realms/TDXConnect/protocol/openid-connect/token"

# pK_ServiceDetails
url = "https://tdx.transportdata.tw/webapi/service?category=basic"
response = requests.get(url)
data = response.json()

pK_ServiceDetails = [item['pK_ServiceDetail'] for item in data['list']]

# 輸出文件夾
OUTPUT_DIR = r'C:\Users\catal\Desktop\py\專案三\data_tdx_csv'

# 創建文件夾
os.makedirs(OUTPUT_DIR, exist_ok=True)

# get data_urls
for pK in pK_ServiceDetails:
    url = f"https://tdx.transportdata.tw/webapi/service/GetServiceDetail/{pK}"
    response = requests.get(url)

    data = response.json()
    data_url = data['dataUrl']
    data_name = data['name']
    if data_url is None or '{' in data_url:
        continue
    print(data_url)

    class Auth():
        def __init__(self, app_id, app_key):
            self.app_id = app_id
            self.app_key = app_key

        def get_auth_header(self):
            content_type = 'application/x-www-form-urlencoded'
            grant_type = 'client_credentials'

            return {
                'content-type': content_type,
                'grant_type': grant_type,
                'client_id': self.app_id,
                'client_secret': self.app_key
            }

    class Data():
        def __init__(self, app_id, app_key, auth_response):
            self.app_id = app_id
            self.app_key = app_key
            self.auth_response = auth_response

        def get_data_header(self):
            auth_JSON = json.loads(self.auth_response.text)
            access_token = auth_JSON.get('access_token')

            return {
                'authorization': 'Bearer ' + access_token
            }

    try:
        a = Auth(app_id, app_key)
        auth_response = requests.post(auth_url, a.get_auth_header())
        d = Data(app_id, app_key, auth_response)
        data_response = requests.get(data_url, headers=d.get_data_header())
    except Exception as e:
        print(f"Error retrieving data from URL: {data_url}\n{str(e)}")
        continue

    # csv
    try:
        data_json = data_response.json()
        filename = f"data_{data_name}.csv"
        file_path = os.path.join(OUTPUT_DIR, filename)
        with open(file_path, 'w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(data_json[0].keys())  # 寫入CSV標頭
            for item in data_json:
                writer.writerow(item.values())  # 寫入CSV數據行
        print(f"Data saved to: {file_path}")
    except Exception as e:
        print(f"Error saving data to CSV file: {str(e)}")
    
    print()


https://tdx.transportdata.tw/api/basic/v2/Basic/City
Data saved to: C:\Users\catal\Desktop\py\專案三\data_tdx_csv\data_全臺縣市代碼資料(City).csv

https://tdx.transportdata.tw/api/basic/v2/Basic/County
Data saved to: C:\Users\catal\Desktop\py\專案三\data_tdx_csv\data_全臺縣市代碼資料(County).csv

https://tdx.transportdata.tw/api/basic/v2/Bus/News/InterCity
Data saved to: C:\Users\catal\Desktop\py\專案三\data_tdx_csv\data_公路客運之最新消息.csv

https://tdx.transportdata.tw/api/basic/v2/Bus/DailyTimeTable/InterCity
Data saved to: C:\Users\catal\Desktop\py\專案三\data_tdx_csv\data_取得公路客運每日營運時刻表資料.csv

https://tdx.transportdata.tw/api/basic/v2/Bus/Operator/InterCity
Data saved to: C:\Users\catal\Desktop\py\專案三\data_tdx_csv\data_公路客運營運業者資料.csv

https://tdx.transportdata.tw/api/basic/v2/Bus/Operator/InterCity
Data saved to: C:\Users\catal\Desktop\py\專案三\data_tdx_csv\data_公路客運營運業者資料.csv

https://tdx.transportdata.tw/api/basic/v2/Bus/Operator/InterCity
Data saved to: C:\Users\catal\Desktop\py\專案三\data_tdx_csv\data_公路客運營運業者資料.csv